# Capstone Project - The Battle of Neighborhoods

# Introduction/Business Problem

I grew up in Taipei, Taiwan and Taipei is the biggest city in Taiwan in terms of population which is about 2.6 million. Taipei is a city that attract lots of tourists annually, and has many attractions for people visiting Taiwan such as one the tallest building Taipei 101. I had the opportunity to study in the United States and San Diego is one of my favorite city in US. The nick name of San Diego is the finest city in the US, attracting hundreds of thousand people every year. I am wondering if a person is debating which cities he should visit and spend one week there, which cities is his final choice.

If a travel agent would like to explore a new market, they city may consider this city’s public transportation, attractions, and room and board. Therefore, I will analyze and compare above mentioned factors and provide a recommendation for this travel agent.

# Data section

I will convert Taipei and San Diego into their equivalent latitude and longitude values, and will use the Foursquare API to explore neighborhoods in both cities. I will use the "explore" function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. I will use the *k*-means clustering algorithm to complete this task. Finally, I will use the Folium library to visualize the neighborhoods in both cities and their emerging clusters. 




In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [2]:
CLIENT_ID = 'FABS5JF1MXSOM4542ZBDJUDHXTMDNFRJAMVAJYTJ3XCJJMHF' # your Foursquare ID
CLIENT_SECRET = 'E1D400E2YXXL1E2JN3K50XCWE52XUDXPXFVHWMG11G2EWFVN' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FABS5JF1MXSOM4542ZBDJUDHXTMDNFRJAMVAJYTJ3XCJJMHF
CLIENT_SECRET:E1D400E2YXXL1E2JN3K50XCWE52XUDXPXFVHWMG11G2EWFVN


In [19]:
#address = '102 North End Ave, New York, NY'
# 2920 Zoo Dr, San Diego, CA 92101
# 7 Xinyi Road, Taipei City, Taiwan 

# address = '2920 Zoo Dr, San Diego, CA '
address = '2920 Zoo Dr, San Diego, CA 92101'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

32.73512015 -117.149244888257


In [20]:
#search_query = 'Italian'
search_query = 'Shopping'
radius = 35000
print(search_query + ' .... OK!')

Shopping .... OK!


In [21]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=FABS5JF1MXSOM4542ZBDJUDHXTMDNFRJAMVAJYTJ3XCJJMHF&client_secret=E1D400E2YXXL1E2JN3K50XCWE52XUDXPXFVHWMG11G2EWFVN&ll=32.73512015,-117.149244888257&v=20180604&query=Shopping&radius=35000&limit=30'

In [22]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5de59cf4fb34b5001b6057e1'},
 'response': {'venues': [{'id': '4d223b0fdd6a236ae00e4638',
    'name': 'Rio Vista Shopping Center',
    'location': {'address': '8310-8740 Rio San Diego Dr',
     'lat': 32.77519254438008,
     'lng': -117.14471720411613,
     'labeledLatLngs': [{'label': 'display',
       'lat': 32.77519254438008,
       'lng': -117.14471720411613}],
     'distance': 4480,
     'postalCode': '92108',
     'cc': 'US',
     'city': 'San Diego',
     'state': 'CA',
     'country': 'United States',
     'formattedAddress': ['8310-8740 Rio San Diego Dr',
      'San Diego, CA 92108',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1fd941735',
      'name': 'Shopping Mall',
      'pluralName': 'Shopping Malls',
      'shortName': 'Mall',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/mall_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1575329029',
    'hasPerk': False},
   

In [23]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",NaN,NaN,NaN,NaN,NaN,NaN,False,4d223b0fdd6a236ae00e4638,8310-8740 Rio San Diego Dr,...,4480,"[8310-8740 Rio San Diego Dr, San Diego, CA 921...","[{'label': 'display', 'lat': 32.77519254438008...",32.775193,-117.144717,NaN,92108,CA,Rio Vista Shopping Center,v-1575329029
1,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",NaN,NaN,NaN,NaN,NaN,NaN,False,4e4ff39afa7626401eb10adc,6348 College Grove Way,...,8798,"[6348 College Grove Way, San Diego, CA 92115, ...","[{'label': 'display', 'lat': 32.74160543917185...",32.741605,-117.055592,NaN,92115,CA,College Grove Shopping Center,v-1575329029
2,"[{'id': '5744ccdfe4b0c0459246b4dc', 'name': 'S...",NaN,NaN,NaN,NaN,NaN,NaN,False,4b8df4d3f964a520761433e3,380 E H St,...,14100,"[380 E H St (at Hidden Vista Dr), Chula Vista,...","[{'label': 'display', 'lat': 32.63970011714374...",32.639700,-117.050271,NaN,91910,CA,Terra Nova Plaza Shopping Center,v-1575329029
3,"[{'id': '4bf58dd8d48988d130941735', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,False,4cbcc9709552b60cbe7be78b,NaN,...,1634,"[San Diego, CA 92103, United States]","[{'label': 'display', 'lat': 32.74925394246662...",32.749254,-117.153988,NaN,92103,CA,Uptown Shopping Center,v-1575329029
4,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",NaN,NaN,NaN,NaN,NaN,NaN,False,465a0609f964a52023471fe3,1640 Camino Del Rio N.,...,3723,[1640 Camino Del Rio N. (btw/ Mission Center R...,NaN,32.768558,-117.148106,NaN,92108,CA,Westfield Mission Valley,v-1575329029


In [24]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Rio Vista Shopping Center,Shopping Mall,8310-8740 Rio San Diego Dr,US,San Diego,United States,NaN,4480,"[8310-8740 Rio San Diego Dr, San Diego, CA 921...","[{'label': 'display', 'lat': 32.77519254438008...",32.775193,-117.144717,NaN,92108,CA,4d223b0fdd6a236ae00e4638
1,College Grove Shopping Center,Shopping Mall,6348 College Grove Way,US,San Diego,United States,NaN,8798,"[6348 College Grove Way, San Diego, CA 92115, ...","[{'label': 'display', 'lat': 32.74160543917185...",32.741605,-117.055592,NaN,92115,CA,4e4ff39afa7626401eb10adc
2,Terra Nova Plaza Shopping Center,Shopping Plaza,380 E H St,US,Chula Vista,United States,at Hidden Vista Dr,14100,"[380 E H St (at Hidden Vista Dr), Chula Vista,...","[{'label': 'display', 'lat': 32.63970011714374...",32.639700,-117.050271,NaN,91910,CA,4b8df4d3f964a520761433e3
3,Uptown Shopping Center,Building,NaN,US,San Diego,United States,NaN,1634,"[San Diego, CA 92103, United States]","[{'label': 'display', 'lat': 32.74925394246662...",32.749254,-117.153988,NaN,92103,CA,4cbcc9709552b60cbe7be78b
4,Westfield Mission Valley,Shopping Mall,1640 Camino Del Rio N.,US,San Diego,United States,btw/ Mission Center Rd & Camino del Este,3723,[1640 Camino Del Rio N. (btw/ Mission Center R...,NaN,32.768558,-117.148106,NaN,92108,CA,465a0609f964a52023471fe3
5,Westfield Horton Plaza,Shopping Mall,324 Horton Plz,US,San Diego,United States,NaN,2568,"[324 Horton Plz, San Diego, CA 92101, United S...","[{'label': 'display', 'lat': 32.71453558118606...",32.714536,-117.161634,NaN,92101,CA,465a059af964a52022471fe3
6,Eastlake Terraces ~ Shopping Center,Shopping Plaza,1480 Eastlake Pkwy,US,Chula Vista,United States,at Olympic Pkwy.,20603,"[1480 Eastlake Pkwy (at Olympic Pkwy.), Chula ...","[{'label': 'display', 'lat': 32.63088198493392...",32.630882,-116.967530,East Chula Vista,91915,CA,4d7441ffd238a1cd5494d40e
7,Southland Plaza Shopping Center,Shopping Mall,Saturn Blvd,US,San Diego,United States,Palm Ave,17522,"[Saturn Blvd (Palm Ave), San Diego, CA 92154, ...","[{'label': 'display', 'lat': 32.58578389959648...",32.585784,-117.090139,NaN,92154,CA,4f636beae4b08b476e63ad76
8,Plaza Sorrento Shopping Center,Shopping Mall,6715 Mira Mesa Blvd,US,San Diego,United States,at Camino Santa Fe,19253,"[6715 Mira Mesa Blvd (at Camino Santa Fe), San...","[{'label': 'display', 'lat': 32.90700965383762...",32.907010,-117.172047,NaN,92121,CA,4c62f6dcde1b2d7f2b93e370
9,Costco,Warehouse Store,2345 Fenton Pkwy,US,San Diego,United States,at Friars Rd,5598,"[2345 Fenton Pkwy (at Friars Rd), San Diego, C...","[{'label': 'display', 'lat': 32.78299155703086...",32.782992,-117.130903,NaN,92108,CA,4a887d3ef964a5209f0620e3


In [25]:
dataframe_filtered.name

0               Rio Vista Shopping Center
1           College Grove Shopping Center
2        Terra Nova Plaza Shopping Center
3                  Uptown Shopping Center
4                Westfield Mission Valley
5                  Westfield Horton Plaza
6     Eastlake Terraces ~ Shopping Center
7         Southland Plaza Shopping Center
8          Plaza Sorrento Shopping Center
9                                  Costco
10          Genesee Plaza Shopping Center
11                        Online Shopping
12           Canyon Plaza Shopping Center
13                              Starbucks
14        La Mesa Springs Shopping Center
15                               Shopping
16         Village Square Shopping Center
17         Convoy Village Shopping Center
18                              Starbucks
19                              Starbucks
20                          Westfield UTC
21            Balboa Mesa Shopping Center
22                Shopping On Garnet Ave!
23                      NEX Navy E

In [28]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map